# Hypothesis Testing: Temperature Effect on Rushing Yards

## Introduction
Testing the hypothesis that: **Rushing yardage is higher in colder temperatures compared to warmer temperatures.**

### Hypothesis Statement
- **Null Hypothesis (H₀)**: [The average rushing yardage in colder temperatures is equal to the average rushing yardage in warmer temperatures.]
- **Alternative Hypothesis (H₁)**: [The average rushing yardage in colder temperatures is greater than the average rushing yardage in warmer temperatures."]

### Purpose
The purpose of this analysis is to investigate the impact of temperature on rushing yardage in NFL games, specifically if colder games result in higher average rushing yardage. This relationship can help provide expectations on team performance and gameplans during colder games. 

### Methodology
To test this hypothesis, we will conduct a [type of test, e.g., independent t-test] using historical NFL game data.
1. **Data Collection**: Gather NFL game data focusing on team rushing yardage and average temperature from each game. 
2. **Data Cleaning & Preparation**: Clean data by adressing missing values and inconsistencies. Categorize the average temperture (e.g., very cold, cold, average, warm, very warm, etc.).
3. **Exploratory Data Analysis (EDA)**: Perform initial analysis to understand distributions of rushing yardage across temperature categories. Create visualizations (box plots, histograms, etc.) to help visualize comparisons.
4. **Hypothesis Testing**: Use an independent t-test to compare average rushing yardage between categorized temperatures. T-test will help determine statistical significance.
5. **Result Interpretation**: Based on the p-value from hypothesis test, determine to accept or reject null hypothesis. Low p-value of (0.05) will indicate the observed difference in rushing yards across temperature categories is statistically significant.
6. **Conclusion**: Interpret results in the context of an NFL game. What can a fan expect from a team and players performance based on weather predictions for the game. 
